In [2]:
from glob import glob
from os.path import isdir
import os
import pandas as pd
abnormal=[]
normal=[]
for x in glob('../data/*/*걷기 무게중심*'):
    abnormal.append(x)
for x in glob('../data/*/*걷기_보통*'):
    normal.append(x)

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
scale_columns=['x','y','z']
print(len(abnormal))
print(len(normal))

40
10


In [6]:
X_train_df=pd.DataFrame()
X_test_df=pd.DataFrame()
y_train=[]
y_test=[]
for f in abnormal:
    df=pd.read_excel(open(f, 'rb'))  
    X_train_df=X_train_df.append(df.loc[:1112,scale_columns],ignore_index=True)
    X_test_df=X_test_df.append(df.loc[1112:1391,scale_columns])
for f in normal:
    df=pd.read_excel(open(f, 'rb'))  
    X_train_df=X_train_df.append(df.loc[:1112,scale_columns])
    X_test_df=X_test_df.append(df.loc[1112:1391,scale_columns])


### X_train, X_test (44520:11131 / 11199: 2800)

In [7]:
X_train_df.head()

,x,y,z
0,26,12416,8896
1,-3685,15424,12160
2,3354,16320,8512
3,602,12544,6208
4,-2341,12800,4672


In [8]:
X_test_df.head()

,x,y,z
1112,1242,11648,2048
1113,602,13760,1920
1114,2010,18048,4224
1115,2010,17472,1792
1116,2842,18560,2816


### y_train, y_test ( 0: abnormal, 1: normal)

In [9]:
y_train=[]
y_test=[]
for i in range(55650):
    if i <=44520:
        y_train.append(0) #비정상
    else:
        y_train.append(1) #정상
for i in range(13999):
    if i <=11199:
        y_test.append(0) #비정상
    else:
        y_test.append(1) #정상

In [10]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

scaler = scaler.fit(X_train_df)

X_train_df = scaler.transform(
  X_train_df.to_numpy()
)

X_test_df= scaler.transform(
  X_test_df.to_numpy()
)

In [11]:
X_train_df.shape

(55650, 3)

In [12]:
import numpy as np
y_train=np.array(y_train).reshape(-1,1)
y_train.shape

(55650, 1)

In [13]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X[i:(i + time_steps)]
        labels = y[i + time_steps]
        Xs.append(v)
        ys.append(labels)
    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [14]:
TIME_STEPS = 233
STEP = 40

X_train, y_train = create_dataset(
    X_train_df,
    y_train,
    TIME_STEPS,
    STEP
)

X_test, y_test = create_dataset(
    X_test_df,
    y_test,
    TIME_STEPS,
    STEP
)

In [15]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1386, 233, 3)
(1386, 1)
(345, 233, 3)
(345, 1)


In [16]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

In [17]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=128,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

In [18]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

Train on 1247 samples, validate on 139 samples
Epoch 1/20
1247/1247 [==============================] - 12s 10ms/sample - loss: 1.9346 - acc: 0.8316 - val_loss: 7.7491 - val_acc: 0.0000e+00
Epoch 2/20
1247/1247 [==============================] - 8s 6ms/sample - loss: 0.5250 - acc: 0.8885 - val_loss: 2.8462 - val_acc: 0.0000e+00
Epoch 3/20
1247/1247 [==============================] - 8s 6ms/sample - loss: 0.4359 - acc: 0.8885 - val_loss: 2.1539 - val_acc: 0.0000e+00
Epoch 4/20
1247/1247 [==============================] - 8s 6ms/sample - loss: 0.4095 - acc: 0.8885 - val_loss: 2.0992 - val_acc: 0.0000e+00
Epoch 5/20
1247/1247 [==============================] - 8s 7ms/sample - loss: 0.4067 - acc: 0.8885 - val_loss: 2.1186 - val_acc: 0.0000e+00
Epoch 6/20
1247/1247 [==============================] - 8s 7ms/sample - loss: 0.3978 - acc: 0.8885 - val_loss: 2.1186 - val_acc: 0.0000e+00
Epoch 7/20
1247/1247 [==============================] - 8s 6ms/sample - loss: 0.3973 - acc: 0.8885 - val_loss: 

In [19]:
y_pred=model.predict(X_test)

In [21]:
model.evaluate(X_test, y_test)

345/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.533135055459064, 0.79710144]

#### To Do - hyperparameter tuning

In [ ]:
from kerastuner.tuners import RandomSearch, BayesianOptimization


tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values